## Calculate Urban Metrics: Fullness
Fraction of the total area of the urban extent that is built-up. Use a custom script that takes the mean value of all the pixels of the GHS built-up within the urban extent. The GHS built-up layer values are expressed as decimals from 0 to 1, and correspond to the fraction of the pixel that is covered by a building (Pesaresi et al., 2016)

In [125]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
import sys, os, inspect, logging, importlib, time

In [127]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math, random

In [128]:
import shapely
from shapely.geometry import mapping
from shapely.geometry import Point

In [129]:
# Get reference to GOSTNets
sys.path.append(r'C:\repos\INFRA_SAP')
from infrasap.urban_metrics import *

In [130]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine # or from affine import Affine

In [131]:
built_up_layer = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\2015_250m_GHS_Built\UZB_2015_GHS_built_merged.tif"

In [132]:
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_FUAs_Project_Mollweide.shp"

In [133]:
input_shapes_gpd.head(5)

,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,...,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,_count,_sum,_mean,sum_sq_km,geometry
0,None,None,None,None,None,None,-1,0,-1,0,...,46.0,19.0,85652.428709,69508.583862,16143.844847,275.0,1021.848162,3.715811,2.221,"POLYGON ((5800000.000 4952000.000, 5798000.000..."
1,None,None,None,None,None,None,-1,0,-1,0,...,18.0,13.0,65461.343613,62013.075363,3448.268250,112.0,482.444814,4.307543,2.680,"POLYGON ((6007000.000 4959000.000, 6008000.000..."
2,None,None,None,None,None,None,-1,0,-1,0,...,178.0,73.0,458838.507896,401135.215576,57703.292320,1086.0,7815.237149,7.196351,4.390,"POLYGON ((6155000.000 4854000.000, 6159000.000..."
3,None,None,None,None,None,None,-1,0,-1,0,...,33.0,16.0,71340.747452,60618.550537,10722.196915,215.0,236.673061,1.100805,7.171,"POLYGON ((4937000.000 5055000.000, 4936000.000..."
4,None,None,None,None,None,None,-1,0,-1,0,...,135.0,58.0,318516.998470,288239.789791,30277.208678,839.0,1557.179394,1.855995,1.153,"POLYGON ((4943000.000 5072000.000, 4944000.000..."


In [134]:
%%time

with rasterio.open(built_up_layer) as src:
    pixelSizeX, pixelSizeY  = src.res
    #print(pixelSizeX, pixelSizeY)
    pixel_area = 250**2
    
    # For each Shape:
        # Select all built-up pixels that are mostly within shape
        # Area of shape = sum of all pixels * area of each pixel
        # Built-up Area = (sum pixel value of each pixel) / 100
        # Fullness index = Built-up Area / Area of Shape

    input_shapes_gpd = gpd.read_file(shpName)

    #for entry in input_shapes_gpd.head(5).iterrows():
    for entry in input_shapes_gpd.iterrows():
        print(entry[0])
        #print(row[1]['geometry'])

        # extract the geometry in GeoJSON format
        geometry = entry[1]['geometry'] # list of shapely geometries
        #geometry = geoms[0] # shapely geometry
        geoms = [mapping(geometry)]

        # extract the raster values values within the polygon 
        out_image, out_transform = mask(src, geoms, crop=True, nodata=-9999.0)
        data = out_image[0,:,:]
        
        row, col = np.where(data != -9999.0) 
        val = np.extract(data != -9999.0, data)

        T1 = out_transform * Affine.translation(0.5, 0.5) # reference the pixel centre
        rc2xy = lambda r, c: (c, r) * T1  
        
        d = gpd.GeoDataFrame({'col':col,'row':row,'val':val})
        
        # coordinate transformation
        d['x'] = d.apply(lambda row: rc2xy(row.row,row.col)[0], axis=1)
        d['y'] = d.apply(lambda row: rc2xy(row.row,row.col)[1], axis=1)
        
        # geometry
        d['geometry'] = d.apply(lambda row: Point(row['x'], row['y']), axis=1)
        
        # Area of shape = sum of all pixels * area of each pixel
        area_of_shape = pixel_area * d.count()[0]
        # Built-up Area = (sum pixel value of each pixel) / 100
        d2 = d.val/100
        built_up = sum(d2) * pixel_area
        # Fullness index = Built-up Area / Area of Shape
        fullness_index = built_up / area_of_shape
        print(f"fullness index: {fullness_index}")
        
        # creates a temporary GDF for just the row's shape
        temp_gdf = input_shapes_gpd.iloc[[entry[0]]]
        
        #print("print temp_gdf")
        #print(temp_gdf)
        
        # Put all metrics in a DataFrame
        metrics_scalar = {}
        metrics_scalar['fullness_index'] = [fullness_index]
        metrics_df = pd.DataFrame(metrics_scalar)
        
        #print("print metrics_scalar")
        #print(metrics_scalar)
        
        # and concatinate it with the row's shape
        new_temp_gdf = pd.concat([temp_gdf.reset_index(drop=True), metrics_df], axis=1)
        
        #print("print new_temp_gdf")
        #print(new_temp_gdf)
        #print(entry[0])
        # put the results of each row into a new DataFrame
        if entry[0] == 0:
            print("new_temp_gdf")
            output_new_temp_gdf = new_temp_gdf
        else:
            output_new_temp_gdf = output_new_temp_gdf.append(new_temp_gdf, ignore_index=True)       

0
fullness index: 0.22897969480799393
new_temp_gdf
1
fullness index: 0.20127188890008557
2


C:\Users\war-machine\Anaconda2\envs\gostnets2\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: Right multiplication will be prohibited in version 3.0


fullness index: 0.314323191100413
3
fullness index: 0.1770401510605442
4
fullness index: 0.1995208518254222
5
fullness index: 0.19116257245749976
6
fullness index: 0.19863370436168215
7
fullness index: 0.13992539984966307
8
fullness index: 0.21638973239737017
9
fullness index: 0.1624956578996489
10
fullness index: 0.19893695507284284
11
fullness index: 0.3534454262900381
12
fullness index: 0.37535855319568395
13
fullness index: 0.3009936120385263
14
fullness index: 0.18361175372358543
15
fullness index: 0.3018655524906334
16
fullness index: 0.21979156901418576
17
fullness index: 0.28764673161235255
18
fullness index: 0.1626766063648292
19
fullness index: 0.19605810806189025
20
fullness index: 0.2670721956893741
21
fullness index: 0.3824447724053254
22
fullness index: 0.2938079453548963
23
fullness index: 0.2611896570969423
24
fullness index: 0.3349508269982114
25
fullness index: 0.19485242555697851
26
fullness index: 0.21054091808586958
27
fullness index: 0.2657242386854582
28
fullness

In [135]:
output_new_temp_gdf.crs

<Projected CRS: ESRI:54009>
Name: World_Mollweide
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Mollweide
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [136]:
# make the GeoDataFrame unprojected
output_new_temp_gdf = output_new_temp_gdf.to_crs('epsg:4326')

In [137]:
# save as shapefile
output_new_temp_gdf.to_file(r"C:\repos\INFRA_SAP\Notebooks\UZB_fullness_index_4326.shp")